## <center>CSE 546: Reinforcement Learning</center>
### <center>Prof. Alina Vereshchaka</center>
#### <center>Spring 2025</center>

Welcome to the Assignment 3, Part 1: Introduction to Actor-Critic Methods! It includes the implementation of simple actor and critic networks and best practices used in modern Actor-Critic algorithms.

## Section 0: Setup and Imports

In [6]:
!pip install swig
!pip install "gymnasium[box2d]"
!pip install gymnasium[atari,accept-rom-license]
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gymnasium as gym
import matplotlib.pyplot as plt
from collections import deque
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils as utils
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379445 sha256=ce3a792dc014019c6288bc0f677d0eb54211433e85174c090ec8d24fb7561b55
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


## Section 1: Actor-Critic Network Architectures and Loss Computation

In this section, you will explore two common architectural designs for Actor-Critic methods and implement their corresponding loss functions using dummy tensors. These architectures are:
- A. Completely separate actor and critic networks
- B. A shared network with two output heads

Both designs are widely used in practice. Shared networks are often more efficient and generalize better, while separate networks offer more control and flexibility.

---


### Task 1a – Separate Actor and Critic Networks with Loss Function

Define a class `SeparateActorCritic`. Your goal is to:
- Create two completely independent neural networks: one for the actor and one for the critic.
- The actor should output a probability distribution over discrete actions (use `nn.Softmax`).
- The critic should output a single scalar value.

 Use `nn.ReLU()` as your activation function. Include at least one hidden layer of reasonable width (e.g. 64 or 128 units).

```python
# TODO: Define SeparateActorCritic class
```

 Next, simulate training using dummy tensors:
1. Generate dummy tensors for log-probabilities, returns, estimated values, and entropies.
2. Compute the actor loss using the advantage (return - value).
3. Compute the critic loss as mean squared error between values and returns.
4. Use a single optimizer for both the Actor and the Critic. In this case, combine the actor and critic losses into a total loss and perform backpropagation.
5. Use a separate optimizers for both the Actor and the Critic. In this case, keep the actor and critic losses separate and perform backpropagation.

```python
# TODO: Simulate loss computation and backpropagation
```

🔗 Helpful references:
- PyTorch Softmax: https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html
- PyTorch MSE Loss: https://pytorch.org/docs/stable/generated/torch.nn.functional.mse_loss.html

---

In [2]:
# TODO: Define a class SeparateActorCritic with separate networks for actor and critic

# BEGIN_YOUR_CODE
class SeparateActorCritic(nn.Module):
    def __init__(self, state_dimension, action_dimension, hidden_size=64):
        super(SeparateActorCritic, self).__init__()
        self.actor_fc1 = nn.Linear(state_dimension, hidden_size)
        self.actor_fc2 = nn.Linear(hidden_size, action_dimension)
        self.actor_softmax = nn.Softmax(dim=-1)
        self.critic_fc1 = nn.Linear(state_dimension, hidden_size)
        self.critic_fc2 = nn.Linear(hidden_size, 1)

    def forward(self, state):
        actor_hidden = F.relu(self.actor_fc1(state))
        action_logits = self.actor_fc2(actor_hidden)
        action_probabilities = self.actor_softmax(action_logits)
        critic_hidden = F.relu(self.critic_fc1(state))
        value = self.critic_fc2(critic_hidden)
        return action_probabilities, value

# END_YOUR_CODE

In [3]:
def simulate_training(model, state):
    action_probabilities, value = model(state)
    dummy_log_probabilities = torch.log(action_probabilities + 1e-8)
    dummy_return_values = torch.randn_like(value)
    dummy_entropies = -torch.sum(action_probabilities * dummy_log_probabilities, dim=1, keepdim=True)
    advantage = dummy_return_values - value.detach()
    actor_loss = -(dummy_log_probabilities * advantage).mean() - 0.01 * dummy_entropies.mean()
    critic_loss = F.mse_loss(value, dummy_return_values)
    total_loss = actor_loss + critic_loss
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    print("Single Optimizer:")
    print("Actor Loss:", actor_loss.item())
    print("Critic Loss:", critic_loss.item())
    print("Total Loss:", total_loss.item())


def simulate_training_separate_optimizers(model, state):
    action_probabilities, value = model(state)
    dummy_log_probabilities = torch.log(action_probabilities + 1e-8)
    dummy_return_values = torch.randn_like(value)
    dummy_entropies = -torch.sum(action_probabilities * dummy_log_probabilities, dim=1, keepdim=True)
    advantage = dummy_return_values - value.detach()
    actor_loss = -(dummy_log_probabilities * advantage).mean() - 0.01 * dummy_entropies.mean()
    critic_loss = F.mse_loss(value, dummy_return_values)
    actor_params = list(model.actor_fc1.parameters()) + list(model.actor_fc2.parameters())
    critic_params = list(model.critic_fc1.parameters()) + list(model.critic_fc2.parameters())
    optimizer_actor = optim.Adam(actor_params, lr=1e-3)
    optimizer_critic = optim.Adam(critic_params, lr=1e-3)
    optimizer_actor.zero_grad()
    actor_loss.backward(retain_graph=True)
    optimizer_actor.step()
    optimizer_critic.zero_grad()
    critic_loss.backward()
    optimizer_critic.step()
    print("\nSeparate Optimizers:")
    print("Actor Loss:", actor_loss.item())
    print("Critic Loss:", critic_loss.item())

In [4]:
state_dimension = 10
action_dimension = 4
batch_size = 5
dummy_state = torch.randn(batch_size, state_dimension)
model = SeparateActorCritic(state_dimension, action_dimension, hidden_size=64)
simulate_training(model, dummy_state)
simulate_training_separate_optimizers(model, dummy_state)

Single Optimizer:
Actor Loss: 0.29983144998550415
Critic Loss: 2.5514073371887207
Total Loss: 2.85123872756958

Separate Optimizers:
Actor Loss: -0.46809422969818115
Critic Loss: 1.2666232585906982


### Discuss the motivation behind each setup and when it may be preferred in practice.

YOUR ANSWER:

In actor–critic reinforcement learning architectures, two distinct components are trained simultaneously: the actor, which learns a policy for selecting actions, and the critic, which estimates the value of states or state–action pairs to guide the actor's learning. When both components are optimized using a single optimizer, the setup remains simple and compact. This approach is especially practical when the actor and critic share a common feature extractor or network trunk. In such cases, a single backward pass can efficiently update all parameters using a combined loss function—typically the sum of the actor’s and critic’s losses. This method is often preferred in simpler environments or during early-stage experimentation, as it reduces implementation complexity and the number of hyperparameters to tune.

However, using separate optimizers for the actor and critic introduces greater flexibility. This setup allows each component to be trained with its own learning rate, optimizer type, or update frequency, which can be crucial in more complex environments. For instance, the critic may require faster learning to provide timely and accurate value estimates, while the actor may benefit from slower, more stable updates to its policy. With separate optimizers, it is easier to manage such divergent learning dynamics without one loss overpowering the other. This method is particularly valuable when the actor and critic do not share network components, or when their losses scale differently over time.

In summary, the single-optimizer approach is generally preferred for its simplicity and efficiency in shared-network settings, while the separate-optimizers setup is advantageous in scenarios demanding more control over the individual learning behaviors of the actor and critic. The choice between the two depends on the complexity of the task, the network architecture, and the desired level of control over training dynamics.

### Task 1b – Shared Network with Actor and Critic Heads + Loss Function

Now define a class `SharedActorCritic`:
- Build a shared base network (e.g., linear layer + ReLU)
- Create two heads: one for actor (output action probabilities) and one for critic (output state value)

```python
# TODO: Define SharedActorCritic class
```

Then:
1. Pass a dummy input tensor through the model to obtain action probabilities and value.
2. Simulate dummy rewards and compute advantage.
3. Compute the actor and critic losses, combine them, and backpropagate.

```python
# TODO: Simulate shared network loss computation and backpropagation
```

 Use `nn.Softmax` for actor output and `nn.Linear` for scalar critic output.

🔗 More reading:
- Policy Gradient Methods: https://spinningup.openai.com/en/latest/algorithms/vpg.html
- Actor-Critic Overview: https://www.tensorflow.org/agents/tutorials/6_reinforce_tutorial
- PyTorch Categorical Distribution: https://pytorch.org/docs/stable/distributions.html#categorical

---

In [5]:
# BEGIN_YOUR_CODE
class SharedActorCritic(nn.Module):
    def __init__(self, state_dimension, action_dimension, hidden_size=64):
        super(SharedActorCritic, self).__init__()
        self.shared_fc = nn.Linear(state_dimension, hidden_size)
        self.actor_fc = nn.Linear(hidden_size, action_dimension)
        self.actor_softmax = nn.Softmax(dim=-1)
        self.critic_fc = nn.Linear(hidden_size, 1)

    def forward(self, state):
        shared_hidden = F.relu(self.shared_fc(state))
        action_logits = self.actor_fc(shared_hidden)
        action_probabilities = self.actor_softmax(action_logits)
        value = self.critic_fc(shared_hidden)
        return action_probabilities, value

# END_YOUR_CODE

In [6]:
def simulate_training_on_shared_network(model, state):
    action_probabilities, value = model(state)
    dummy_log_probabilities = torch.log(action_probabilities + 1e-8)
    dummy_return_values = torch.randn_like(value)
    advantage = dummy_return_values - value.detach()
    dummy_entropies = -torch.sum(action_probabilities * dummy_log_probabilities, dim=1, keepdim=True)
    actor_loss = -(dummy_log_probabilities * advantage).mean() - 0.01 * dummy_entropies.mean()
    critic_loss = F.mse_loss(value, dummy_return_values)
    total_loss = actor_loss + critic_loss
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    print("Shared Network Training:")
    print("Actor Loss:", actor_loss.item())
    print("Critic Loss:", critic_loss.item())
    print("Total Loss:", total_loss.item())

In [7]:
state_dimension = 10
action_dimension = 4
batch_size = 5
dummy_state = torch.randn(batch_size, state_dimension)
model = SharedActorCritic(state_dimension, action_dimension, hidden_size=64)
simulate_training_on_shared_network(model, dummy_state)

Shared Network Training:
Actor Loss: -0.4370136857032776
Critic Loss: 0.8117109537124634
Total Loss: 0.3746972680091858


### Discuss the motivation behind each setup and when it may be preferred in practice.

YOUR ANSWER:

The shared actor–critic architecture employs a single “backbone” network to extract features from each state, which are then consumed by both the policy (actor) and the value function (critic). By sharing this feature extractor, the critic’s gradient signals naturally inform the actor about which aspects of the state space are most relevant, often yielding faster and more sample‑efficient learning. This parameter‐and‐compute efficiency proves particularly advantageous in low‑data regimes or on resource‑constrained devices. However, when the objectives of policy improvement and value estimation diverge substantially, the shared backbone can introduce destructive interference—at which point fully separate networks may be more appropriate.

From the shared representation, the actor head produces logits for each discrete action and applies a softmax to obtain a valid probability distribution. Operating in logit space simplifies the incorporation of temperature scaling for exploration control and the computation of entropy regularization to discourage premature convergence to deterministic policies. In domains that require continuous control, this softmax head would be replaced by a Gaussian parameterization (learning both means and variances), while very large discrete action spaces might leverage sampling approximations such as Gumbel‑Softmax to contain computational costs.

Simultaneously, the critic head maps the shared features to a single scalar value. Training proceeds by minimizing the mean‑squared error between this estimate and the empirically observed return. To guide the actor, an advantage is computed by subtracting the critic’s detached estimate from the actual return, supplying a lower‑variance policy gradient without bias. In particularly noisy or outlier‑prone environments, practitioners may swap the MSE loss for a Huber loss to improve robustness, and employ multi‑step returns or Generalized Advantage Estimation (GAE) to refine the bias–variance characteristics of the advantage signal.

An entropy bonus—scaled in this implementation by a factor of 0.01—is added to the actor’s loss to maintain sufficient exploration throughout training. Early in learning, a higher entropy coefficient can encourage broader policy coverage, whereas annealing it toward zero supports convergence to the highest‑reward behaviors. This straightforward mechanism helps navigate the exploration–exploitation trade‑off, especially in sparse‑reward or deceptive environments.

Finally, actor and critic losses are summed into a single optimization objective and updated jointly using Adam at a modest learning rate of 1 × 10⁻³. Joint backpropagation through the shared backbone streamlines implementation and leverages coupled gradient information. For large‑scale or highly sensitive tasks, however, one might adopt separate optimizers or learning‑rate schedules for each head, introduce gradient clipping to prevent instability, or revert to RMSProp—as seen in A2C/A3C implementations—if it yields superior empirical performance.

In practice, the choice between full sharing, partial sharing, or complete decoupling of actor and critic networks depends on factors such as compute budget, data availability, and the degree to which policy and value estimation benefit from common features. A fully shared network with MSE critic loss and a modest entropy bonus serves as a robust default; more advanced scenarios may demand tailored advantage estimators, robust critic losses, or even distributional critics to capture richer value information.

## Section 2: Auto-Adaptive Network Setup for Environments

You will now create a function that builds a shared actor-critic network that adapts to any Gymnasium environment. This function should inspect the environment and build input/output layers accordingly.

### Task 2: Auto-generate Input and Output Layers
Write a function `create_shared_network(env)` that constructs a neural network using the following rules:
- The input layer should match the environment's observation space.
- The output layer for the **actor** should depend on the action space:
  - For discrete actions: output probabilities using `nn.Softmax`.
  - For continuous actions: output mean and log std for a Gaussian distribution.
- The **critic** always outputs a single scalar value.

```python
# TODO: Define function `create_shared_network(env)`
```

#### Environments to Support:
Test your function with the following environments:
1. `CliffWalking-v0` (Use one-hot encoding for discrete integer observations.)
2. `LunarLander-v3` (Standard Box space for observations and discrete actions.)
3. `PongNoFrameskip-v4` (Use gym wrappers for Atari image preprocessing.)
4. `HalfCheetah-v5` (Continuous observation and continuous action.)

```python
# TODO: Loop through environments and test `create_shared_network`
```

Hint: Use `gym.spaces` utilities to determine observation/action types dynamically.

🔗 Observation/Action Space Docs:
- https://gymnasium.farama.org/api/spaces/

---

In [24]:
!pip install "gym[atari,accept-rom-license]" ale-py autorom
!AutoROM --accept-license

  Using cached ale_py-0.8.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached AutoROM-0.4.2-py3-none-any.whl.metadata (2.8 kB)
Using cached ale_py-0.8.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.10.2
    Uninstalling ale-py-0.10.2:
      Successfully uninstalled ale-py-0.10.2
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires ale-py>=0.10.1, but you have ale-py 0.8.1 which is incompatible.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


AutoROM will download the Atari 2600 ROMs.
They will be installed to:
	/usr/local/lib/python3.11/dist-packages/AutoROM/roms

Existing ROMs will be overwritten.
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/adventure.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/air_raid.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/alien.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/amidar.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/assault.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/asterix.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/asteroids.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/atlantis.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/atlantis2.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/backgammon.bin
Installed /usr/local/lib/python3.11/dist-packages/AutoROM/roms/bank_heist.bin
Inst

In [3]:
from gym.envs.registration import registry
pong_envs = [env_id for env_id in registry.keys() if "Pong" in env_id]
print("Pong env IDs:", pong_envs)

Pong env IDs: ['ALE/Pong-v5', 'ALE/Pong-ram-v5', 'Pong-v0', 'PongDeterministic-v0', 'PongNoFrameskip-v0', 'Pong-v4', 'PongDeterministic-v4', 'PongNoFrameskip-v4', 'Pong-ram-v0', 'Pong-ramDeterministic-v0', 'Pong-ramNoFrameskip-v0', 'Pong-ram-v4', 'Pong-ramDeterministic-v4', 'Pong-ramNoFrameskip-v4']


In [4]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F

class Actor_Critic_Network(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_size=128, is_continuous=False):
        super(Actor_Critic_Network, self).__init__()
        self.is_continuous = is_continuous
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.actor = nn.Linear(hidden_size, output_dim if not is_continuous else hidden_size)
        self.critic = nn.Linear(hidden_size, 1)
        if is_continuous:
            self.log_std = nn.Parameter(torch.zeros(output_dim))

    def forward(self, x):
        x = F.relu(self.fc1(x))
        value = self.critic(x)
        if self.is_continuous:
            mean = self.actor(x)
            return (mean, self.log_std.exp()), value
        else:
            logits = self.actor(x)
            probs = F.softmax(logits, dim=-1)
            return probs, value

In [5]:
def create_shared_network(env, hidden_size=128):
    observation_space = env.observation_space
    action_space = env.action_space
    if isinstance(observation_space, gym.spaces.Discrete):
        input_dim = observation_space.n
    else:
        input_dim = int(torch.prod(torch.tensor(observation_space.shape)))
    is_continuous = isinstance(action_space, gym.spaces.Box)
    output_dim = action_space.shape[0] if is_continuous else action_space.n
    return Actor_Critic_Network(input_dim, output_dim, hidden_size, is_continuous)

In [6]:
environment_names = [
    "PongNoFrameskip-v4",
]

for environment_name in environment_names:
    try:
        env = gym.make(environment_name)
        print(f"\nCreating network for {environment_name}:")
        model = create_shared_network(env, hidden_size=128)
    except Exception as e:
        print(f"Failed to create environment {environment_name}: {e}")
        continue
    if isinstance(env.observation_space, gym.spaces.Discrete):
        dummy_input = torch.nn.functional.one_hot(torch.tensor([0, 1, 2]), num_classes=env.observation_space.n).float()
    elif isinstance(env.observation_space, gym.spaces.Box):
        shape = env.observation_space.shape
        input_dim = int(torch.prod(torch.tensor(shape)))
        dummy_input = torch.randn(3, input_dim)
    else:
        print("Unsupported observation space.")
        continue
    output = model(dummy_input)
    if isinstance(env.action_space, gym.spaces.Discrete):
        action_probabilities, value = output
        print("Actor Output (Action Probabilities):", action_probabilities)
        print("Critic Output (State Value):", value)
    elif isinstance(env.action_space, gym.spaces.Box):
        (mean, std), value = output
        print("Actor Output (Mean):", mean)
        print("Actor Output (Std):", std)
        print("Critic Output (State Value):", value)


Creating network for PongNoFrameskip-v4:
Actor Output (Action Probabilities): tensor([[0.1646, 0.1336, 0.0849, 0.1604, 0.2226, 0.2339],
        [0.1832, 0.1227, 0.1208, 0.1979, 0.1802, 0.1952],
        [0.1405, 0.1414, 0.0984, 0.2110, 0.2078, 0.2008]],
       grad_fn=<SoftmaxBackward0>)
Critic Output (State Value): tensor([[-0.4015],
        [-0.0230],
        [-0.1364]], grad_fn=<AddmmBackward0>)


In [9]:
!pip install "gymnasium[mujoco]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 17.6 MB/s eta 0:00:00


In [10]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F

class SharedNetwork(nn.Module):
    def __init__(self, input_dim, action_space, hidden_size=128):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.critic = nn.Linear(hidden_size, 1)
        if isinstance(action_space, gym.spaces.Discrete):
            self.actor = nn.Linear(hidden_size, action_space.n)
            self.is_discrete = True
        elif isinstance(action_space, gym.spaces.Box):
            self.actor_mean = nn.Linear(hidden_size, action_space.shape[0])
            self.actor_log_std = nn.Parameter(torch.zeros(action_space.shape[0]))
            self.is_discrete = False
        else:
            raise NotImplementedError("Unsupported action space type")

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        value = self.critic(x)
        if self.is_discrete:
            logits = self.actor(x)
            action_probabilities = F.softmax(logits, dim=-1)
            return action_probabilities, value
        else:
            mean = self.actor_mean(x)
            log_std = self.actor_log_std.expand_as(mean)
            return (mean, log_std), value

def create_shared_network(env, hidden_size=128):
    observation_space = env.observation_space
    if isinstance(observation_space, gym.spaces.Discrete):
        input_dim = observation_space.n
    elif isinstance(observation_space, gym.spaces.Box):
        input_dim = int(torch.tensor(observation_space.shape).prod().item())
    else:
        raise NotImplementedError("Unsupported observation space type")
    return SharedNetwork(input_dim, env.action_space, hidden_size)

environment_names = [
    "CliffWalking-v0",
    "LunarLander-v3",
    "PongNoFrameskip-v4",
    "HalfCheetah-v5",
]

for environment_name in environment_names:
    try:
        env = gym.make(environment_name)
        print(f"\n Creating network for {environment_name}:")
        model = create_shared_network(env, hidden_size=128)
    except Exception as e:
        print(f" Failed to create environment {environment_name}: {e}")
        continue
    if isinstance(env.observation_space, gym.spaces.Discrete):
        dummy_input = F.one_hot(torch.tensor([0, 1, 2]), num_classes=env.observation_space.n).float()
    elif isinstance(env.observation_space, gym.spaces.Box):
        obs_shape = env.observation_space.shape
        input_dim = int(torch.tensor(obs_shape).prod().item())
        dummy_input = torch.randn(3, input_dim)
    else:
        print(f"Unsupported observation space for {environment_name}")
        continue
    output = model(dummy_input)
    if isinstance(env.action_space, gym.spaces.Discrete):
        action_probabilities, value = output
        print("Actor Output (Action Probabilities):", action_probabilities)
        print("Critic Output (State Value):", value)
    elif isinstance(env.action_space, gym.spaces.Box):
        (mean, log_std), value = output
        print("Actor Output (Mean):", mean)
        print("Actor Output (Log Std):", log_std)
        print("Critic Output (State Value):", value)


 Creating network for CliffWalking-v0:
Actor Output (Action Probabilities): tensor([[0.2469, 0.2396, 0.2684, 0.2451],
        [0.2505, 0.2377, 0.2719, 0.2399],
        [0.2515, 0.2371, 0.2715, 0.2399]], grad_fn=<SoftmaxBackward0>)
Critic Output (State Value): tensor([[0.0211],
        [0.0357],
        [0.0219]], grad_fn=<AddmmBackward0>)

 Creating network for LunarLander-v3:
Actor Output (Action Probabilities): tensor([[0.2293, 0.2333, 0.3125, 0.2250],
        [0.2376, 0.2296, 0.3096, 0.2232],
        [0.2317, 0.2284, 0.2949, 0.2450]], grad_fn=<SoftmaxBackward0>)
Critic Output (State Value): tensor([[0.0298],
        [0.0094],
        [0.0268]], grad_fn=<AddmmBackward0>)
 Failed to create environment PongNoFrameskip-v4: Environment `PongNoFrameskip` doesn't exist.

 Creating network for HalfCheetah-v5:
Actor Output (Mean): tensor([[ 0.0841,  0.0689,  0.0924,  0.1195,  0.1081, -0.1889],
        [ 0.2681,  0.2020,  0.0368,  0.0860,  0.1245, -0.1484],
        [-0.0117,  0.0318,  0.0853

### Discuss the motivation behind each setup and when it may be preferred in practice.

YOUR ANSWER:

The first implementation presents a streamlined actor–critic agent capable of handling both discrete and continuous action spaces with minimal ceremony. It employs a single hidden layer to transform the environment’s state into a shared feature vector, which then branches into two heads: one producing a scalar value estimate (the critic) and the other yielding action parameters (the actor). In discrete settings, the actor head emits logits that are converted to probabilities via softmax; in continuous domains, it predicts Gaussian means alongside a learned log‐standard‐deviation. By consolidating actor and critic into a single network and keeping the architecture shallow, this design minimizes code complexity and parameter count, making it particularly well suited for low‑dimensional, vector‑based tasks where rapid prototyping is paramount.

A companion helper function automatically introspects the Gym environment’s observation and action spaces to infer input and output dimensions. Discrete observations are one‑hot encoded, Box observations are flattened into vectors, and action dimensions are read directly from the environment’s API. This meta‑programming approach allows the same class and helper to instantiate agents for a variety of tasks—from tabular CliffWalking to continuous control benchmarks—without any manual reshaping or boilerplate. The printed dummy outputs serve as a sanity check, verifying that the network’s input and output shapes align with the environment, though they also underscore a limitation: flattening raw pixel inputs ignores spatial structure, which is critical for vision‑based domains like Atari.

The second implementation, built atop Gymnasium, extends representational capacity by adding a second shared hidden layer. This deeper backbone is better equipped to capture the nonlinear dynamics typical of more challenging environments, such as MuJoCo’s HalfCheetah‑v5. It retains the same automatic dimension‑inference logic, preserving the plug‑and‑play convenience of the first implementation. For continuous actions, a global log‑standard‑deviation parameter ensures stable yet tunable exploration, while discrete actions continue to leverage a softmax head over learned logits.

In practical terms, the choice among these architectures depends on the complexity of the task and the computational budget available. For simple, low‑dimensional problems—where the state space can be effectively summarized by a shallow multilayer perceptron—the single‑layer network offers unmatched simplicity and speed. When richer features are required to model complex dynamics, the two‑layer shared network strikes a favorable balance between expressivity and ease of use. And when dealing with high‑dimensional visual inputs, practitioners are advised to replace the MLP encoder with convolutional or residual architectures to exploit spatial hierarchies, recognizing that deeper backbones are essential for extracting meaningful features from images.

### Task 3: Write Observation Normalization Function
Create a function `normalize_observation(obs, env)` that:
- Checks if the observation space is `Box` and has `low` and `high` attributes.
- If so, normalize the input observation.
- Otherwise, return the observation unchanged.

```python
# TODO: Define `normalize_observation(obs, env)`
```

Test this function with observations from:
- `LunarLander-v3`
- `PongNoFrameskip-v4`

Note: Atari observations are image arrays. Normalize pixel values to [0, 1]. For LunarLander-v3, the different elements in the observation vector have different ranges. Normalize them to [0, 1] using the `low` and `high` attributes of the observation space.


---

In [10]:
import gymnasium as gymn
import gym
import numpy as np

def normalize_observation(obs, env):
    if isinstance(obs, dict):
        return {k: normalize_observation(v, env) for k, v in obs.items()}
    if isinstance(obs, (list, tuple)):
        try:
            arrays = [np.array(o) for o in obs]
            shapes = [a.shape for a in arrays]
            if len(set(shapes)) == 1:
                obs = np.stack(arrays, axis=0)
            else:
                return type(obs)(normalize_observation(o, env) for o in obs)
        except Exception:
            return type(obs)(normalize_observation(o, env) for o in obs)
    try:
        obs = np.asarray(obs, dtype=np.float32)
    except Exception as e:
        raise ValueError(f"Unable to convert observation to numpy array: {e}")
    if hasattr(env.observation_space, 'low') and hasattr(env.observation_space, 'high'):
        if hasattr(env.observation_space, 'dtype') and env.observation_space.dtype == np.uint8:
            normalized = obs / 255.0
        else:
            low = env.observation_space.low.astype(np.float32)
            high = env.observation_space.high.astype(np.float32)
            normalized = (obs - low) / (high - low)
        normalized = np.clip(normalized, 0, 1)
        return normalized
    else:
        return obs

In [11]:
env_lunar = gymn.make("LunarLander-v3")
obs_lunar, _ = env_lunar.reset()
norm_obs_lunar = normalize_observation(obs_lunar, env_lunar)
print("LunarLander-v3 original observation:", obs_lunar)
print("LunarLander-v3 normalized observation:", norm_obs_lunar)

LunarLander-v3 original observation: [ 0.00670757  1.412558    0.6793884   0.07277823 -0.00776559 -0.1538915
  0.          0.        ]
LunarLander-v3 normalized observation: [0.5013415  0.7825116  0.5339694  0.5036389  0.49938202 0.49230543
 0.         0.        ]


In [12]:
env_pong = gym.make("PongNoFrameskip-v4")
obs_pong = env_pong.reset()
norm_obs_pong = normalize_observation(obs_pong, env_pong)
if isinstance(norm_obs_pong, (list, tuple)):
    shapes = [np.asarray(frame).shape for frame in norm_obs_pong]
    mins = [np.min(frame) for frame in norm_obs_pong]
    maxs = [np.max(frame) for frame in norm_obs_pong]
    print("PongNoFrameskip-v4 normalized observation shapes:", shapes)
    print("PongNoFrameskip-v4 normalized observation value ranges:")

    for i, (mi, ma) in enumerate(zip(mins, maxs)):
        print(f"  Frame {i}: {mi} to {ma}")

else:
    print("PongNoFrameskip-v4 original observation shape:", np.asarray(obs_pong).shape)
    print("PongNoFrameskip-v4 normalized observation range:",
          norm_obs_pong.min(), "to", norm_obs_pong.max())

PongNoFrameskip-v4 normalized observation shapes: [(210, 160, 3), ()]
PongNoFrameskip-v4 normalized observation value ranges:
  Frame 0: 0.0 to 0.8941176533699036
  Frame 1: {'lives': np.float32(0.0), 'episode_frame_number': np.float32(0.0), 'frame_number': np.float32(0.0)} to {'lives': np.float32(0.0), 'episode_frame_number': np.float32(0.0), 'frame_number': np.float32(0.0)}


### Discuss the motivation behind each setup and when it may be preferred in practice.

YOUR ANSWER:

The normalization function in the provided setup is designed with consistency and robustness in mind. Environments from Gym and Gymnasium often deliver observations in diverse formats—ranging from raw pixel arrays to structured dictionaries or flat numerical vectors. To handle this variability, the function recursively traverses through each component of the observation, converting elements into standardized NumPy arrays of float32 data type. This ensures that regardless of the structure—be it a simple array or a nested dictionary—the output is uniform and compatible with downstream neural network inputs. As a result, model pipelines become more reliable and less prone to data-type-related errors.

Once the observation is cast into a suitable numeric format, the function proceeds to normalize it within the
0
,

1
0, 1 range. If the environment provides explicit low and high bounds through its observation space, a linear transformation is applied to each element, mapping the minimum to 0 and the maximum to 1. This step is critical for learning stability: without normalization, features with varying scales can produce erratic gradient updates, which may lead to unstable or slow training. By ensuring all features lie within the same range, the model benefits from smoother optimization dynamics and improved convergence behavior.

In scenarios involving image-based observations—such as in Atari environments like Pong—where the data is typically in uint8 format (ranging from 0 to 255), the function simply rescales pixel values by dividing by 255.0. This transformation, while simple, is a standard preprocessing step in computer vision tasks and enables agents to process visual data in a more numerically stable form. Though more advanced techniques like grayscale conversion or frame stacking may be used later in the pipeline, this basic normalization offers a practical baseline for quick prototyping and testing.

Overall, this normalization strategy is especially useful during experimentation and early-stage development. It offers a generalized preprocessing approach that accommodates a wide variety of environments, from classic control tasks like LunarLander to more complex vision-based games like Pong. By abstracting away the intricacies of observation formats and scaling, researchers and practitioners can concentrate on refining learning algorithms without being encumbered by input inconsistencies.










## Section 4: Gradient Clipping

To prevent exploding gradients, it's common practice to clip gradients before optimizer updates.

### Task 4: Clip Gradients for Actor-Critic Networks
Use dummy tensors and apply gradient clipping with the following PyTorch method:
```python
# During training, after loss.backward():
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
```

Reuse the loss computation from Task 1a or 1b. After computing the gradients, apply gradient clipping.
Print the gradient norm before and after clipping to verify it’s applied.

🔗 PyTorch Docs: https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html


---

In [3]:
# BEGIN_YOUR_CODE
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

class DummyActorCritic(nn.Module):
    def __init__(self, input_size, num_actions):
        super(DummyActorCritic, self).__init__()
        self.fc = nn.Linear(input_size, 128)
        self.actor = nn.Linear(128, num_actions)
        self.critic = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc(x))
        action_probabilities = F.softmax(self.actor(x), dim=-1)
        value = self.critic(x)
        return action_probabilities, value

# END_YOUR_CODE

In [4]:
def simulate_training_with_grad_clip(model, state):
    action_probabilities, value = model(state)
    dummy_log_probabilities = torch.log(action_probabilities + 1e-8)
    dummy_return_values = torch.randn_like(value)
    dummy_entropies = -torch.sum(action_probabilities * dummy_log_probabilities, dim=1, keepdim=True)
    advantage = dummy_return_values - value.detach()
    actor_loss = -(dummy_log_probabilities * advantage).mean() - 0.01 * dummy_entropies.mean()
    critic_loss = F.mse_loss(value, dummy_return_values)
    total_loss = actor_loss + critic_loss
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    optimizer.zero_grad()
    total_loss.backward()
    pre_clip_norm = 0.0

    for p in model.parameters():
        if p.grad is not None:
            pre_clip_norm += p.grad.data.norm(2).item() ** 2

    pre_clip_norm = pre_clip_norm ** 0.5
    clip_norm = utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
    post_clip_norm = 0.0

    for p in model.parameters():
        if p.grad is not None:
            post_clip_norm += p.grad.data.norm(2).item() ** 2

    post_clip_norm = post_clip_norm ** 0.5
    optimizer.step()
    print("Single Optimizer Training Step:")
    print("Actor Loss:", actor_loss.item())
    print("Critic Loss:", critic_loss.item())
    print("Total Loss:", total_loss.item())
    print("Gradient norm before clipping:", pre_clip_norm)
    print("Gradient norm reported by clip_grad_norm_ (before clipping):", clip_norm)
    print("Gradient norm after clipping:", post_clip_norm)

In [7]:
input_size = 8
num_actions = 4
model = DummyActorCritic(input_size, num_actions)
state = torch.randn(1, input_size)
simulate_training_with_grad_clip(model, state)

Single Optimizer Training Step:
Actor Loss: -2.9831323623657227
Critic Loss: 4.527118682861328
Total Loss: 1.5439863204956055
Gradient norm before clipping: 23.084273351243823
Gradient norm reported by clip_grad_norm_ (before clipping): tensor(23.0843)
Gradient norm after clipping: 0.4999999788338938


### Discuss the motivation behind each setup and when it may be preferred in practice.

YOUR ANSWER:

In high‑variance settings—particularly deep or recurrent architectures used in reinforcement learning—it is not uncommon for gradient magnitudes to spike unexpectedly, destabilizing training or even producing NaNs. Gradient clipping addresses this by capping the global ℓ₂ norm of all gradients (0.5 in the provided example), ensuring that no single update can become excessively large. This safeguard allows practitioners to maintain an aggressive learning rate without fearing that an isolated gradient explosion will derail the entire training process.

The DummyActorCritic architecture exemplifies a widely adopted design in actor–critic methods: a shared “backbone” network that projects raw observations into a 128‑dimensional feature space, which then bifurcates into two distinct heads—one estimating the policy and the other estimating state value. Sharing these lower‑level parameters not only reduces the overall model size but also fosters beneficial inductive biases, since the critic’s insights about state quality can inform the actor’s action preferences, and vice versa. Empirically, this parameter sharing often accelerates convergence and stabilizes learning.

For discrete action domains, the actor head’s raw outputs are converted into a valid probability distribution via softmax. The core policy‑gradient objective—the negative expected log‑probability of chosen actions weighted by the estimated advantage—drives the agent toward actions yielding higher long‑term returns. To prevent premature convergence to a narrow set of actions, a modest entropy bonus is incorporated into the loss. This term rewards uncertainty, encouraging the policy to continue exploring until sufficient evidence accumulates for confident decision‑making.

Rather than managing separate optimizers or learning‑rate schedules for the actor and critic, the example consolidates all parameters under a single Adam optimizer. This unified approach simplifies hyperparameter tuning and implicitly couples the learning rates of both heads. During early experimentation or prototyping—when actor and critic losses typically remain on comparable scales—this simplicity often outweighs the benefits of decoupling. Only when one loss term begins to dominate does the case for multiple optimizers become compelling.

Before integrating with a real environment or computing actual Monte Carlo returns, the pipeline undergoes a “smoke test” using randomly generated dummy returns. By executing a full forward pass, computing both actor and critic losses, applying gradient clipping, and stepping the optimizer on synthetic data, this initial check verifies that tensor dimensions, gradient flows, clipping logic, and optimizer calls are all implemented correctly. Identifying and resolving these foundational issues on noise alone is far more efficient than debugging them amidst the complexity of live roll‑outs.

If you are working in a team, provide a contribution summary.
| Team Member | Step# | Contribution (%) |
|---|---|---|
|   | Task 1 |   |
|   | Task 2 |   |
|   | Task 3 |   |
|   | Task 4 |   |
|   | **Total** |   |


Shishir Hebbar - Task 1,2,3,4 - 50% <br>
Shaurya Mathur - Task 1,2,3,4 - 50%

#### Refrences:

1) https://www.geeksforgeeks.org/actor-critic-algorithm-in-reinforcement-learning/ <br>
2) https://www.gymlibrary.dev/environments/toy_text/cliff_walking/ <br>
3) https://www.gymlibrary.dev/environments/box2d/lunar_lander/ <br>
4) https://huggingface.co/ThomasSimonini/ppo-PongNoFrameskip-v4 <br>
5) https://www.gymlibrary.dev/environments/mujoco/half_cheetah/